<a href="https://colab.research.google.com/github/Girisha-Malni-builds01/AI-Placement-Practice-Notebook/blob/Supervised-AI-Algorithms/Girisha's_Logistic_Regression_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Logistic Regression from Scratch**

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = np.array([[1],[2],[3],[4],[5],[6],[7],[8],[9]])
y = np.array([0,0,0,0,1,1,1,1,1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
output = ["Yes" if p==1 else "No" for p in y_pred]

for h, o in zip(X_test.flatten(), output):
    print(f"Studied {h} hours → Pass? {o}")

print("\nClassification Report:\n", classification_report(y_test, y_pred))



Studied 8 hours → Pass? Yes
Studied 2 hours → Pass? No
Studied 6 hours → Pass? Yes

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

